# Centrevo Analysis Tutorial

This notebook demonstrates how to use Centrevo's Python interface for population genetics analysis.

## Installation

```bash
pip install centrevo pyarrow matplotlib pandas
```

In [ ]:
import centrevo
from centrevo.plotting import (
    plot_nucleotide_composition,
    plot_distance_matrix,
    export_to_pyarrow_table
)
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

## 1. Create a Population

First, let's create an initial population with centromeric repeat structure.

In [ ]:
# Define repeat structure
alphabet = centrevo.Alphabet.dna()
base_a = centrevo.Nucleotide.A()

structure = centrevo.RepeatStructure(
    alphabet=alphabet,
    init_base=base_a,
    ru_length=171,      # Repeat unit length
    rus_per_hor=12,     # Repeat units per higher-order repeat
    hors_per_chr=50,    # Higher-order repeats per chromosome
    chrs_per_hap=1,     # Chromosomes per haplotype
)

# Create population
pop = centrevo.create_initial_population(size=50, structure=structure)

print(f"Created population with {pop.size()} individuals")
print(f"Generation: {pop.generation()}")
print(f"Chromosome length: {structure.chr_length()} bp")

## 2. Calculate Diversity Metrics

Calculate standard population genetics diversity metrics.

In [ ]:
# Calculate metrics
pi = centrevo.nucleotide_diversity(pop, chromosome_idx=0)
tajima_d = centrevo.tajimas_d(pop, chromosome_idx=0)
theta_w = centrevo.wattersons_theta(pop, chromosome_idx=0)
hap_div = centrevo.haplotype_diversity(pop, chromosome_idx=0)

# Display results
metrics_df = pd.DataFrame({
    'Metric': ['Nucleotide diversity (π)', 'Tajima\'s D', 'Watterson\'s θ', 'Haplotype diversity'],
    'Value': [pi, tajima_d, theta_w, hap_div]
})

print("\nDiversity Metrics:")
print(metrics_df.to_string(index=False))

## 3. Export to PyArrow and DataFrame

Export metrics in a format that's easy to work with.

In [ ]:
# Export metrics
metrics_dict = centrevo.export_diversity_metrics(pop, chromosome_idx=0)

# Convert to DataFrame (in practice, you'd collect over multiple generations)
df = pd.DataFrame([metrics_dict])
print("\nExported Metrics DataFrame:")
print(df)

## 4. Nucleotide Composition Analysis

In [ ]:
# Calculate composition
comp = centrevo.nucleotide_composition(pop, None, None, None)

# Plot
fig = plot_nucleotide_composition(comp, title="Population Nucleotide Composition")
plt.show()

# GC content
gc = centrevo.gc_content(pop, None, None, None)
print(f"\nPopulation GC content: {gc:.4f}")

## 5. Linkage Disequilibrium Analysis

In [ ]:
# Calculate LD between two positions
ld_stats = centrevo.linkage_disequilibrium(
    pop, pos1=1000, pos2=5000,
    chromosome_idx=0, haplotype_idx=0
)

if ld_stats:
    print("LD Statistics (positions 1000 vs 5000):")
    print(f"  D: {ld_stats['D']:.6f}")
    print(f"  D': {ld_stats['D_prime']:.6f}")
    print(f"  r²: {ld_stats['r_squared']:.6f}")
else:
    print("Could not calculate LD (no variation at these sites)")

## 6. Distance Matrix Analysis

In [ ]:
# Calculate distance matrix (using subset for visualization)
# Note: For large populations, this can be memory intensive
matrix = centrevo.distance_matrix(pop, chromosome_idx=0)

print(f"Distance matrix shape: {len(matrix)}×{len(matrix[0])}")
print(f"(2n sequences: {pop.size()} individuals × 2 haplotypes)")

# Plot heatmap
fig = plot_distance_matrix(matrix, figsize=(10, 8))
plt.show()

## 7. Export Distance Matrix to DataFrame

Convert distance matrix to long format for analysis.

In [ ]:
# Export in long format
dist_data = centrevo.export_distance_matrix(pop, chromosome_idx=0)
dist_df = pd.DataFrame(dist_data)

print("Distance DataFrame (first 10 rows):")
print(dist_df.head(10))

# Summary statistics
print("\nDistance summary:")
print(dist_df['distance'].describe())

## 8. Polymorphism Analysis

In [ ]:
# Count segregating sites
seg_sites = centrevo.count_segregating_sites(pop, chromosome_idx=0, haplotype_idx=0)
seq_length = structure.chr_length()

print(f"Segregating sites: {seg_sites}")
print(f"Sequence length: {seq_length}")
print(f"Proportion polymorphic: {seg_sites/seq_length:.6f}")

## 9. Summary

Create a comprehensive summary table.

In [ ]:
summary = pd.DataFrame({
    'Metric': [
        'Population size',
        'Sequence length',
        'Nucleotide diversity (π)',
        'Tajima\'s D',
        'Watterson\'s θ',
        'Haplotype diversity',
        'GC content',
        'Segregating sites',
    ],
    'Value': [
        pop.size(),
        seq_length,
        f"{pi:.6f}",
        f"{tajima_d:.6f}",
        f"{theta_w:.6f}",
        f"{hap_div:.6f}",
        f"{gc:.4f}",
        seg_sites,
    ]
})

print("\n" + "="*50)
print("Population Genetics Summary")
print("="*50)
print(summary.to_string(index=False))

## Next Steps

- **Time series analysis**: Collect metrics over multiple generations
- **Comparative analysis**: Compare multiple populations
- **Custom visualizations**: Use pandas/matplotlib for advanced plots
- **Statistical tests**: Perform hypothesis testing on the data

See the [Python README](python/README.md) for more examples and documentation.